# Five Thirty Eight Hidden API 

Investigating the change of Bidens approval ratings in relation to key events in American Politics

In [1]:
import requests
import pandas as pd
import altair as alt
import statsmodels.api as sm

### Sending request to 538 using Hidden API

In [2]:
url = 'https://projects.fivethirtyeight.com/biden-approval-rating/polls.json'


response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code}")
    print(response.text)


In [3]:
#seperating the useful data and saving it as a dataframe called Biden_Opinion_Polls_df

Biden_Opinion_Polls= [{'startDate': poll['startDate'], 'approve': poll['answers'][0]['pct'], 'disapprove': poll['answers'][1]['pct']} for poll in data]

Biden_dataframe= pd.DataFrame(Biden_Opinion_Polls)

Biden_dataframe.to_csv('../Data/Biden_Opinion_Polls_df.csv', index=False)


### Finding important events in US politics 

I used the following websites to find a list of the biggest news stories throughout Bidens Presidency to create a list of events:

1. https://edition.cnn.com/2022/12/31/us/top-100-digital-stories-2022-trnd/index.html 
2. https://scrippsnews.com/stories/looking-back-at-the-biggest-headlines-of-2023/

In [4]:
# creating and saving the event list dataframe 

events_data = pd.DataFrame({
    'Date': ['2021-06-16', '2021-03-11', '2021-11-06', '2021-08-26', '2022-03-08', '2022-08-24','2022-02-26','2023-05-11','2023-09-14','2023-11-14','2023-07-2','2023-01-07'],
    'Event': ['Geneva Summit with Putin', 'American Rescue Plan', 'Infrastructure Investment and Jobs Act Signed', 'Hurricane Ida', 'Ban on Russian energy imports', 'Student Loan forgiveness plan','Trump sides with Putin as Biden tries to stop a war','Title 42 expires as federal government tries to handle migration','Hunter Biden indicted by federal prosecutors on gun charges','Biden met with Chinese leader Xi Jinping','Interest rates jump to highest levels in 22 years','Republicans take majority in the House of Representatives for the first time in four years'],})

events_data.to_csv('../Data/event_data.csv', index=False)


### Creating Graph using Vega-Altaire

We used chatGBT to help us add smoothed lines of best fit as we were not able to do this

In [5]:


# Assume 'Biden_dataframe' is your DataFrame
df_long = pd.melt(Biden_dataframe, id_vars=['startDate'], value_vars=['approve', 'disapprove'],
                  var_name='Opinion', value_name='Percentage')

# Altair Chart for points with filled circles
chart_points = alt.Chart(df_long).mark_point(size=50, opacity=0.25, filled=True).encode(
    x=alt.X('startDate:T', axis=alt.Axis(tickCount='month', format='%b %Y', labelAngle=-90)),
    y=alt.Y('Percentage:Q', axis=alt.Axis(title='Percentage')),
    color=alt.Color('Opinion:N', scale=alt.Scale(domain=['approve', 'disapprove'], range=['green', 'red']), legend=alt.Legend(title='Opinion')),
).properties(
    title='Biden Approval Rates Overtime',
    width=600,
    height=400
)

# Adding smoothed lines of best fit using Chatgbt 

loess_smoothed = df_long.groupby(['Opinion', 'startDate']).mean().reset_index()
loess_smoothed['smoothed'] = loess_smoothed.groupby('Opinion')['Percentage'].transform(lambda x: sm.nonparametric.lowess(x, range(len(x)), frac=0.3)[:, 1])

chart_lines = alt.Chart(loess_smoothed).mark_line(color='black').encode(
    x=alt.X('startDate:T', axis=alt.Axis(tickCount='month', format='%b %Y', labelAngle=-90), title='Date'),
    y=alt.Y('smoothed:Q', axis=alt.Axis(title='Percentage')),
    detail='Opinion:N'
)


# adding event lines 
vertical_lines = alt.Chart(events_data).mark_rule(color='blue', strokeWidth=1.5,opacity=0.5).encode(
    x='Date:T',
    tooltip=['Event:N'],
)

# adding all the componenets of the graph together
Biden_Approval_Rates_Overtime = alt.layer(chart_points, chart_lines, vertical_lines).interactive()


Biden_Approval_Rates_Overtime


alt.LayerChart(...)